# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686581


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:27,  3.11s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:00,  2.22s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:42,  1.62s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:29,  1.18s/it]

...

Rendering models:  80%|████████  | 24/30 [00:09<00:01,  4.85it/s]

Rendering models:  87%|████████▋ | 26/30 [00:09<00:00,  5.57it/s]

Rendering models:  90%|█████████ | 27/30 [00:09<00:00,  5.91it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  7.65it/s]

CThomas                               0.000810
not-logged-in-db23dc032fc427a4e4d9    0.000474
thedarktree                           0.001038
acohen12                              0.000588
justsomenoodles                       0.000961
jmfranci                              0.001470
aioeu                                 0.000491
mbaker23                              0.002434
not-logged-in-ef704c739eb8412f355e    0.000895
msalin03                              0.000782
pollux2728                            0.003821
danielrwik                            0.000636
carolynjoy                            0.004795
schultzk                              0.002443
not-logged-in-314d48142a3a122ba666    0.001552
Hollywood1116                         0.001598
Evan.wickham                          1.219581
Planetme                              0.002491
41Letters                             0.000561
zzw                                   0.001742
not-logged-in-6d3e4eb12f2149553110    0.000472
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())